In [4]:
from PIL import Image
from pillow_heif import register_heif_opener
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [5]:
# Creates page number

class Incrementer:
    def __init__(self, start=1):
        self.value = start
    
    def get_next(self):
        current = self.value
        self.value += 1
        return f"{current}"  # Format the string to include "Page " before the number

# Example usage
incrementer = Incrementer()

In [6]:
##### Title Page

doc = Document()
sec_pr = doc.sections[0]._sectPr 

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")

# Set font properties
title_run.bold = True
title_run.font.size = Pt(36)
title_run.font.name = 'Times New Roman'

# Set paragraph alignment to center
title_paragraph.alignment = 1  # Center alignment

#Add space after title paragraph
space_paragraph = doc.add_paragraph()
space_paragraph.paragraph_format.space_before = Pt(6)

# Define containers for user input
client = input("Enter the client's name: ")
property_address = input("Enter the property address: ")
report_date = input("Enter the report date (e.g., YYYY-MM-DD): ")
prg_ref_num = input("Enter the PRG reference number: ")

# Create a table with two columns and four rows
table = doc.add_table(rows=4, cols=2)

# Define the left column values and corresponding user input variables
left_column_values = ["CLIENT", "PROPERTY ADDRESS", "REPORT DATE", "PRG REFERENCE NUMBER"]
right_column_values = [client, property_address, report_date, prg_ref_num]

# Populate the table
for i, row in enumerate(table.rows):
    cell_left = row.cells[0]
    cell_right = row.cells[1]

    # Set the text for the left column
    cell_left.text = left_column_values[i]
    cell_left.paragraphs[0].runs[0].font.size = Pt(12)
    cell_left.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_left.paragraphs[0].runs[0].font.bold = True
    cell_left.paragraphs[0].runs[0].text = cell_left.paragraphs[0].runs[0].text.upper()

    # Set the text for the right column
    cell_right.text = right_column_values[i]
    cell_right.paragraphs[0].runs[0].font.size = Pt(12)
    cell_right.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_right.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.RIGHT
    
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Register HEIC opener with Pillow
register_heif_opener()

# Function to convert HEIC to JPG
def convert_heic_to_jpg(heic_file_path, jpg_file_path):
    # Open HEIC file using Pillow
    image = Image.open(heic_file_path)
    
    # Save as JPG
    image.save(jpg_file_path, format='JPEG')

# File paths
heic_file_path = 'IMG_1303.HEIC'
jpg_file_path = 'IMG_1303.jpg'

# Convert HEIC to JPG
convert_heic_to_jpg(heic_file_path, jpg_file_path)

# Add space after the table
doc.add_paragraph()  # Add a blank paragraph for spacing

# Create a new table with one cell to hold the image
image_table = doc.add_table(rows=1, cols=1)
image_cell = image_table.cell(0, 0)

# Add the image to the table cell
image_cell.paragraphs[0].add_run().add_picture(jpg_file_path, width=Inches(6.0))  # Adjust width as needed

# Create a new paragraph for the address
address_paragraph = doc.add_paragraph()
address_paragraph.alignment = 1  # Center alignment

# Add address text to the paragraph
address_run = address_paragraph.add_run(property_address)

# Set font properties for the address
address_run.bold = True
address_run.italic = True
address_run.font.size = Pt(12)
address_run.font.name = 'Times New Roman'

# Add the PRG Logo image to the same paragraph
logo_paragraph = doc.add_paragraph()
logo_paragraph.alignment = 1  # Center alignment

# Add the PRG Logo image to the paragraph
logo_paragraph.add_run().add_picture('PRG Logo.jpg', width=Inches(2.0))  # Adjust width as needed

# Add custom text and page number to footer with Calibri font and size 8
for section in doc.sections:
    footer = section.footer
    
    # Add the custom text to the footer
    p = footer.add_paragraph()
    p.alignment = 1  # Center alignment
    run = p.add_run("Pugh Real Estate Group, LLC\n")
    run.font.name = 'Calibri'
    run.font.size = Pt(8)

    run = p.add_run("Copyright © 2024\n")
    run.font.name = 'Calibri'
    run.font.size = Pt(8)

    run = p.add_run("Confidential – Do not copy or redistribute without Permission from client\n\n")
    run.font.name = 'Calibri'
    run.font.size = Pt(8)

    # Add page number with the text "Page "
    run = p.add_run(f"Page {incrementer.get_next()}")
    run.font.name = 'Calibri'
    run.font.size = Pt(8)
    
# create new borders element
pg_borders = OxmlElement('w:pgBorders')
# specifies how the relative positioning of the borders should be calculated
pg_borders.set(qn('w:offsetFrom'), 'page')
for border_name in ('top', 'left', 'bottom', 'right',): # set all borders
    border_el = OxmlElement(f'w:{border_name}')
    border_el.set(qn('w:val'), 'single') # a single line
    border_el.set(qn('w:sz'), '4') # for meaning of  remaining attrs please look docs
    border_el.set(qn('w:space'), '24')
    border_el.set(qn('w:color'), 'auto')
    pg_borders.append(border_el) # register single border to border el
sec_pr.append(pg_borders) # apply border changes to section

# Save the document
doc.save("appraisal_report.docx")

Enter the client's name:  sdf
Enter the property address:  sdfg
Enter the report date (e.g., YYYY-MM-DD):  sdfg
Enter the PRG reference number:  sdfg
